<a href="https://colab.research.google.com/github/javier-jaime/Tool-Crib/blob/master/LangChain/QA_Chain_Neo4j_Cypher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GraphCypherQAChain


This notebook shows how to use an LLMs to provide a natural language interface to a graph database you can query with the Cypher query language.

In [ ]:
# Install necessary Libraries
!pip install openai
!pip install neo4j-driver
!pip install langchain
from neo4j import GraphDatabase, basic_auth
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00


In [ ]:
# Access your Secret Key and declare it as an environmental variable
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
# Initialize the Neo4j Driver
graph = Neo4jGraph(
    url="bolt://3.231.56.117:7687", username="neo4j", password="progress-charge-chief"
)

In [ ]:
# Refresh Graph Schema (if required)
graph.refresh_schema()
print(graph.get_schema)


        Node properties are the following:
        [{'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'born', 'type': 'INTEGER'}], 'labels': 'Person'}, {'properties': [{'property': 'tagline', 'type': 'STRING'}, {'property': 'title', 'type': 'STRING'}, {'property': 'released', 'type': 'INTEGER'}], 'labels': 'Movie'}]
        Relationship properties are the following:
        [{'type': 'ACTED_IN', 'properties': [{'property': 'roles', 'type': 'LIST'}]}, {'type': 'REVIEWED', 'properties': [{'property': 'summary', 'type': 'STRING'}, {'property': 'rating', 'type': 'INTEGER'}]}]
        The relationships are the following:
        ['(:Person)-[:ACTED_IN]->(:Movie)', '(:Person)-[:DIRECTED]->(:Movie)', '(:Person)-[:PRODUCED]->(:Movie)', '(:Person)-[:WROTE]->(:Movie)', '(:Person)-[:FOLLOWS]->(:Person)', '(:Person)-[:REVIEWED]->(:Movie)']
        


In [ ]:
# Initialize the QA Chain
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

## Prompt/Query the Neo4j Sandbox

In [ ]:
prompt = input('Enter your question: ')
chain.run(prompt)

Enter your question: Who directed Forrest Gump?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person)-[:DIRECTED]->(m:Movie {title: 'Forrest Gump'}) RETURN DISTINCT labels(m) as Type, m.title as Title, m.released as Released, m.tagline as Tagline, m.summary as Summary, m.rating as Rating, m.roles as Roles, m.born as Born, m.name as Name
Full Context:
[]

> Finished chain.


'Robert Zemeckis.'